##### Copyright 2019 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Decodificar archivos DICOM para imágenes médicas

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/dicom"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/dicom.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/dicom.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a> </td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/io/tutorials/dicom.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a> </td>
</table>

## Descripción general

En este tutorial se muestra cómo usar `tfio.image.decode_dicom_image` en TensorFlow IO para decodificar archivos DICOM con TensorFlow.

## Configuración y uso

#### Descargar la imagen DICOM

La imagen DICOM que se usa en este tutorial proviene del [conjunto de datos de radiografías de tórax de los Institutos Nacionales de Salud](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest).

El conjunto de datos de radiografías de tórax de los Institutos Nacionales de Salud (NIH, por sus siglas en inglés) consta de 100.000 imágenes no identificadas de radiografías de tórax en formato PNG, proporcionadas por el Centro Clínico de los NIH y que se pueden descargar a través de [este enlace](https://nihcc.app.box.com/v/ChestXray-NIHCC).

Google Cloud también proporciona una versión DICOM de las imágenes, disponible en [Cloud Storage](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest).

En este tutorial, descargará un archivo de muestra del conjunto de datos del [repositorio de GitHub.](https://github.com/tensorflow/io/raw/master/docs/tutorials/dicom/dicom_00000001_000.dcm)

Nota: Para obtener más información sobre el conjunto de datos, consulte la siguiente referencia:

- Xiaosong Wang, Yifan Peng, Le Lu, Zhiyong Lu, Mohammadhadi Bagheri, Ronald Summers, ChestX-ray8: Hospital-scale Chest X-ray Database and Benchmarks on Weakly-Supervised Classification and Localization of Common Thorax Diseases, IEEE CVPR, pp. 3462-3471, 2017


In [ ]:
!curl -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/dicom/dicom_00000001_000.dcm
!ls -l dicom_00000001_000.dcm

### Instale los paquetes necesarios y reinicie el tiempo de ejecución

In [ ]:
try:
  # Use the Colab's preinstalled TensorFlow 2.x
  %tensorflow_version 2.x 
except:
  pass

In [ ]:
!pip install tensorflow-io

### Decodifique la imagen DICOM

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

In [ ]:
import tensorflow_io as tfio

image_bytes = tf.io.read_file('dicom_00000001_000.dcm')

image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)

skipped = tfio.image.decode_dicom_image(image_bytes, on_error='skip', dtype=tf.uint8)

lossy_image = tfio.image.decode_dicom_image(image_bytes, scale='auto', on_error='lossy', dtype=tf.uint8)


fig, axes = plt.subplots(1,2, figsize=(10,10))
axes[0].imshow(np.squeeze(image.numpy()), cmap='gray')
axes[0].set_title('image')
axes[1].imshow(np.squeeze(lossy_image.numpy()), cmap='gray')
axes[1].set_title('lossy image');

### Decodifique los metadatos DICOM y trabaje con etiquetas

`decode_dicom_data` decodifica la información de la etiqueta. `dicom_tags` contiene información útil como la edad y el sexo del paciente, por lo que puede usar etiquetas DICOM como `dicom_tags.PatientsAge` y `dicom_tags.PatientsSex`. tensorflow_io toma prestada la misma notación de etiquetas del paquete pydicom dicom.

In [ ]:
tag_id = tfio.image.dicom_tags.PatientsAge
tag_value = tfio.image.decode_dicom_data(image_bytes,tag_id)
print(tag_value)

In [ ]:
print(f"PatientsAge : {tag_value.numpy().decode('UTF-8')}")

In [ ]:
tag_id = tfio.image.dicom_tags.PatientsSex
tag_value = tfio.image.decode_dicom_data(image_bytes,tag_id)
print(f"PatientsSex : {tag_value.numpy().decode('UTF-8')}")

## Documentación

Este paquete tiene dos operaciones que envuelven funciones `DCMTK`. `decode_dicom_image` decodifica los datos de píxeles de los archivos DICOM y `decode_dicom_data` decodifica la información de las etiquetas. `tags` contiene etiquetas DICOM útiles como `tags.PatientsName`. La notación de etiquetas se toma prestada del paquete [`pydicom`](https://pydicom.github.io/) dicom.

### Obtener datos de imagen DICOM

```python
io.dicom.decode_dicom_image(
    contents,
    color_dim=False,
    on_error='skip',
    scale='preserve',
    dtype=tf.uint16,
    name=None
)
```

- **`contents`**: un tensor de tipo cadena de texto. De dimensión 0. El archivo DICOM codificado con cadena de bytes
- **`color_dim`**: un `bool` opcional. El valor predeterminado es `False`. Si es `True`, se agregará un tercer canal a todas las imágenes lo que formará un tensor tridimensional. Una imagen en escala de grises de 1024 x 1024 será 1024 x 1024 x 1
- **`on_error`**: El valor predeterminado es `skip`. Este atributo establece el comportamiento en caso de que ocurra un error al abrir la imagen o si el tipo de salida no puede acomodar todos los valores de entrada posibles. Por ejemplo, si el usuario establece el tipo de salida en `tf.uint8`, pero una imagen dicom almacena un tipo `tf.uint16`. `strict` arroja un error. `skip` devuelve un tensor vacío unidimensional. `lossy` continúa con la operación escalando el valor a través del atributo `scale`.
- **`scale`**: Valores predeterminados para `preserve`. Este atributo establece qué hacer con la escala de los valores de entrada. `auto` escalará automáticamente los valores de entrada, si el tipo de salida es entero, `auto` usará la escala de salida máxima, por ejemplo, un `uint8` que almacena valores de [0, 255] se puede estirar linealmente para llenar un `uint16` que es [0,65535]. Si la salida es flotante, `auto` escalará a [0,1]. `preserve` mantiene los valores como están, un valor de entrada mayor que la salida máxima posible se recortará.
- **`dtype`**: un `tf.DType` opcional de: `tf.uint8, tf.uint16, tf.uint32, tf.uint64, tf.float16, tf.float32, tf.float64`. El valor predeterminado es `tf.uint16`.
- **`name`**: un nombre para la operación (opcional).

**Devuelve** un `Tensor` de tipo `dtype` y la forma está determinada por el archivo DICOM.

### Obtener datos de etiquetas DICOM

```python
io.dicom.decode_dicom_data(
    contents,
    tags=None,
    name=None
)
```

- **`contents`**: un tensor de tipo cadena de texto. De dimensión 0. El archivo DICOM codificado con cadena de bytes
- **`tags`**: un tensor de tipo `tf.uint32` de cualquier dimensión. Estos números `uint32` se asignan directamente a etiquetas DICOM
- **`name`**: un nombre para la operación (opcional).

**Devuelve** un `Tensor` de tipo `tf.string` y la misma forma que `tags`. Si una etiqueta dicom es una lista de cadenas, se combinan en una sola cadena y se separan por una doble barra invertida `**Devuelve** un `Tensor` de tipo `tf.string` y la misma forma que `tags`. Si una etiqueta dicom es una lista de cadenas, se combinan en una sola cadena y se separan por una doble barra invertida . Hay un error en [DCMTK](https://support.dcmtk.org/docs/) si la etiqueta es una lista de números, solo el elemento cero se devolverá como una cadena.

### Bibtex

Si este paquete le sirvió, use lo siguiente para citarlo:

```
@misc{marcelo_lerendegui_2019_3337331,
  author       = {Marcelo Lerendegui and
                  Ouwen Huang},
  title        = {Tensorflow Dicom Decoder},
  month        = jul,
  year         = 2019,
  doi          = {10.5281/zenodo.3337331},
  url          = {https://doi.org/10.5281/zenodo.3337331}
}
```

### Licencia

Copyright 2019 Marcelo Lerendegui, Ouwen Huang, Gradient Health Inc.

Bajo licencia de Apache, Versión 2.0 (la "Licencia"); no puede usar este archivo excepto de conformidad con la Licencia. Puede obtener una copia de la Licencia en

http://www.apache.org/licenses/LICENSE-2.0

A menos que lo exija la ley aplicable o se acuerde por escrito, el software distribuido bajo la Licencia se distribuye "TAL CUAL", SIN GARANTÍAS NI CONDICIONES DE NINGÚN TIPO, ya sean expresas o implícitas. Consulte la Licencia para conocer el idioma específico que rige los permisos y limitaciones de la Licencia.